In [2]:
import pandas as pd
import numpy as np
import os
import csv
import arcpy
from scipy.interpolate import griddata
from arcpy import env

In [3]:
# Load the data
filtered_df = pd.read_csv("8-updated_all.csv")
C2VSimFG_df = pd.read_csv("C:/c2vsimfg1.5/C2VSimFG_node_layering_assigned.csv")

# Create the new column by averaging INTERVALSTART and INTERVALEND
filtered_df['INTERVALLENGTH'] = (filtered_df['INTERVALEND'] + filtered_df['INTERVALSTART'])


# Calculate total records and number of unique values
total_records = len(filtered_df)

# Find number of unique values in the WCRNUMBER column
unique_wcrnumber = filtered_df['WellName'].nunique()

# Print the result
# Print total records and unique values per column
print(f"Total records: {total_records}")
print(f"Number of unique WCRNUMBER values: {unique_wcrnumber}")

filtered_df.head()

C:\Users\betebari\AppData\Local\Temp\ipykernel_34284\4069548150.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  filtered_df = pd.read_csv("8-updated_all.csv")


Total records: 7903497
Number of unique WCRNUMBER values: 310272


,WellName,Well,Point,X,Y,Zland,INTERVALSTART,INTERVALEND,Coarse,Kxy,SY,Ss,Kv,INTERVAL_MIDPOINT,INTERVALLENGTH
0,WCR2023-007196,307088,1,525069.817561,4.461330e+06,927.0387436,0.0,15.0,17.5,0.050500,6.500,0.050500,0.0005,7.5,15.0
1,WCR2023-007196,307088,2,525069.817561,4.461330e+06,927.0387436,15.0,220.0,44.0,21.260410,6.500,0.002693,0.0025,117.5,235.0
2,WCR2017-004848,290543,1,530512.503921,4.481274e+06,1354.260214,0.0,65.0,90.0,217.519700,19.000,0.000051,0.5000,32.5,65.0
3,WCR2017-004848,290543,2,530512.503921,4.481274e+06,1354.260214,65.0,200.0,45.0,21.260424,5.045,0.002525,0.0025,132.5,265.0
4,WCR2022-011845,304223,1,531436.615099,4.483232e+06,1443.940007,0.0,38.0,90.0,217.519700,19.000,0.000051,0.5000,19.0,38.0


In [4]:
# Drop rows where AverageCoarseFraction is NaN
filtered_df = filtered_df.dropna(subset=['Coarse'])


# Calculate total records and number of unique values
total_records = len(filtered_df)

# Find number of unique values in the WCRNUMBER column
unique_wcrnumber = filtered_df['WellName'].nunique()

# Print total records and unique values per column
print(f"Total records: {total_records}")
print(f"Number of unique WCRNUMBER values: {unique_wcrnumber}")

# Print the result
filtered_df.head()

Total records: 7894214
Number of unique WCRNUMBER values: 310202


,WellName,Well,Point,X,Y,Zland,INTERVALSTART,INTERVALEND,Coarse,Kxy,SY,Ss,Kv,INTERVAL_MIDPOINT,INTERVALLENGTH
0,WCR2023-007196,307088,1,525069.817561,4.461330e+06,927.0387436,0.0,15.0,17.5,0.050500,6.500,0.050500,0.0005,7.5,15.0
1,WCR2023-007196,307088,2,525069.817561,4.461330e+06,927.0387436,15.0,220.0,44.0,21.260410,6.500,0.002693,0.0025,117.5,235.0
2,WCR2017-004848,290543,1,530512.503921,4.481274e+06,1354.260214,0.0,65.0,90.0,217.519700,19.000,0.000051,0.5000,32.5,65.0
3,WCR2017-004848,290543,2,530512.503921,4.481274e+06,1354.260214,65.0,200.0,45.0,21.260424,5.045,0.002525,0.0025,132.5,265.0
4,WCR2022-011845,304223,1,531436.615099,4.483232e+06,1443.940007,0.0,38.0,90.0,217.519700,19.000,0.000051,0.5000,19.0,38.0


# Apply Interpoltion method on C2VSimFG L1

In [6]:
# Set the workspace
env.workspace = "C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params"
env.overwriteOutput = True

In [7]:
import os
from arcpy.sa import Spline

# Set the environment to allow overwriting existing files
arcpy.env.overwriteOutput = True

# Define the coordinate system (UTM Zone 10N)
spatial_ref = arcpy.SpatialReference(26910)

# Check if Spatial Analyst extension is available and check it out
if arcpy.CheckExtension("Spatial") == "Available":
    arcpy.CheckOutExtension("Spatial")
    print("Spatial Analyst extension checked out")
else:
    print("Spatial Analyst extension not available")
    raise RuntimeError("Spatial Analyst extension is required but not available.")

try:
    # Filter the DataFrame for relevant data
    coords_df = C2VSimFG_df[['X', 'Y', 'L1', 'L2', 'L3', 'L4']]

    # Save the filtered DataFrame to a CSV file for ArcPy use
    coords_csv = "C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params/C2VSimFG_coords_4layers.csv"
    coords_df.to_csv(coords_csv, index=False)
    
    # Create the geodatabase if it doesn't exist
    gdb_path = "C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params/C2VSimFG_points_4layers.gdb"
    if not arcpy.Exists(gdb_path):
        arcpy.management.CreateFileGDB("C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params", "C2VSimFG_points_4layers.gdb")
        print(f"Created geodatabase: {gdb_path}")
    else:
        print(f"Geodatabase already exists: {gdb_path}")
    
    # Create a point feature class from the CSV
    point_fc = f"{gdb_path}/point_fc"
    arcpy.management.XYTableToPoint(coords_csv, point_fc, 'X', 'Y', coordinate_system=spatial_ref)

    # Define layers and output raster paths
    layers = ['L1', 'L2', 'L3', 'L4']
    output_dir = "C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params/"
    cell_size = 500  # Adjust based on your required grid resolution
    
    for layer in layers:
        print(f"Processing interpolation for {layer}...")
        
        # Define temporary and final raster outputs
        temp_raster_output = f"in_memory/{layer}_temp"
        final_raster_output = os.path.join(output_dir, f"{layer}_interpolated.tif")
        
        # Append a suffix if the raster already exists
        suffix = 1
        while arcpy.Exists(final_raster_output) or os.path.exists(final_raster_output):
            final_raster_output = os.path.join(output_dir, f"{layer}_interpolated_{suffix}.tif")
            suffix += 1
            print(f"Assigned new name to avoid overwrite: {final_raster_output}")

        # Perform spline interpolation
        spline_result = Spline(point_fc, layer, cell_size)
        
        # Save to temporary workspace
        spline_result.save(temp_raster_output)
        print(f"Temporary raster saved for {layer}")
        
        # Copy the temporary raster to the desired location
        arcpy.management.CopyRaster(temp_raster_output, final_raster_output)
        print(f"Spline interpolation complete for {layer}. Raster saved at {final_raster_output}")
    
except arcpy.ExecuteError as e:
    print(f"ArcPy ExecuteError: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")
finally:
    # Check-in Spatial Analyst extension after completion
    arcpy.CheckInExtension("Spatial")
    print("Spatial Analyst extension checked in")


Spatial Analyst extension checked out
Geodatabase already exists: C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params/C2VSimFG_points_4layers.gdb
Processing interpolation for L1...
Temporary raster saved for L1
Spline interpolation complete for L1. Raster saved at C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params/L1_interpolated.tif
Processing interpolation for L2...
Temporary raster saved for L2
Spline interpolation complete for L2. Raster saved at C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params/L2_interpolated.tif
Processing interpolation for L3...
Temporary raster saved for L3
Spline interpolation complete for L3. Raster saved at C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params/L3_interpolated.tif
Processing interpolation for L4...
Temporary raster saved for L4
Spline interpolation complete for L4. Raster saved at C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params/L4_interpolated.tif
Spatial Analyst extension checked in


In [8]:
# Ensure 'X' and 'Y' represent the spatial coordinates
array = np.array(np.rec.fromrecords(filtered_df.values, names=filtered_df.columns.tolist()))

# Define the output feature class and spatial reference (use the appropriate reference for your data)
output_fc = "C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params/C2VSimFG_points_L1.gdb/Filterd_WCRs_AEM"
spatial_ref = arcpy.SpatialReference(26910)  # Replace with your actual spatial reference (UTM Zone 10N)

# Check if the feature class already exists, and if so, delete it
if arcpy.Exists(output_fc):
    arcpy.management.Delete(output_fc)

# Convert the DataFrame (now a NumPy array) back to a feature class
arcpy.da.NumPyArrayToFeatureClass(array, output_fc, ['X', 'Y'], spatial_ref)

print(f"Feature class saved as: {output_fc}")

Feature class saved as: C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params/C2VSimFG_points_L1.gdb/Filterd_WCRs_AEM


# ExtractValuesToPoints operation(Interpolated C2VSimFG-L1 Thickness) to  points ((Filtered_WCRs_n_AEM ))

In [10]:
from arcpy.sa import ExtractValuesToPoints

# Check out the Spatial Analyst extension
if arcpy.CheckExtension("Spatial") == "Available":
    arcpy.CheckOutExtension("Spatial")
    print("Spatial Analyst extension checked out")
else:
    print("Spatial Analyst extension not available. Exiting.")
    raise RuntimeError("Spatial Analyst extension is required but not available.")

try:
    # Set workspace and allow overwriting
    arcpy.env.workspace = "C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params/C2VSimFG_points_L1.gdb"
    arcpy.env.overwriteOutput = True

    # Define inputs
    input_points = "C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params/C2VSimFG_points_L1.gdb/Filterd_WCRs_AEM"
    if not arcpy.Exists(input_points):
        raise FileNotFoundError(f"Input points dataset '{input_points}' does not exist.")

    gdb_path = "C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params/C2VSimFG_points_L1.gdb"

    # Define layers and output folder
    layers = ['L1', 'L2', 'L3', 'L4']
    raster_folder = "C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params/"
    output_folder = gdb_path

    # Loop through each layer
    for layer in layers:
        print(f"Processing raster extraction for {layer}...")

        # Define input raster for the current layer
        input_raster = f"{raster_folder}/{layer}_interpolatd.tif"
        if not arcpy.Exists(input_raster):
            print(f"Raster for {layer} not found: {input_raster}. Skipping...")
            continue

        # Define output feature class for the current layer
        output_fc = f"{output_folder}/Filterd_WCRs_AEM_output_{layer}"

        # Validate inputs
        print(f"Validating inputs for {layer}...")
        print(f"Points: {arcpy.GetCount_management(input_points)[0]} features")
        print(f"Raster: {arcpy.Describe(input_raster).spatialReference.name}")
        print(f"Points projection: {arcpy.Describe(input_points).spatialReference.name}")

        # Extract raster values to points
        ExtractValuesToPoints(
            in_point_features=input_points,
            in_raster=input_raster,
            out_point_features=output_fc,
            interpolate_values="NONE",
            add_attributes="VALUE_ONLY"
        )
        print(f"Raster values from {input_raster} extracted to {output_fc}")

except arcpy.ExecuteError as e:
    print(f"ArcPy ExecuteError: {e}")
except FileNotFoundError as e:
    print(f"FileNotFoundError: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")
finally:
    arcpy.CheckInExtension("Spatial")
    print("Spatial Analyst extension checked in")

Spatial Analyst extension checked out
Processing raster extraction for L1...
Validating inputs for L1...
Points: 7894214 features
Raster: NAD_1983_UTM_Zone_10N
Points projection: NAD_1983_UTM_Zone_10N
Raster values from C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params//L1_interpolatd.tif extracted to C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params/C2VSimFG_points_L1.gdb/Filterd_WCRs_AEM_output_L1
Processing raster extraction for L2...
Raster for L2 not found: C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params//L2_interpolatd.tif. Skipping...
Processing raster extraction for L3...
Raster for L3 not found: C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params//L3_interpolatd.tif. Skipping...
Processing raster extraction for L4...
Raster for L4 not found: C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params//L4_interpolatd.tif. Skipping...
Spatial Analyst extension checked in


In [37]:
# Path to the CSV file (replace this with your actual file)
csv_file = "C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params/Filtered_WCRs_AEM.csv"

# Path to the geodatabase
gdb_path = "C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params/C2VSimFG_points_4layers.gdb"

# Output feature class name
output_fc = f"{gdb_path}/Filtered_WCRs_AEM"

# Define spatial reference (UTM Zone 10N)
spatial_ref = arcpy.SpatialReference(26910)

# Create the point feature class from CSV
arcpy.management.XYTableToPoint(
    in_table=csv_file,
    out_feature_class=output_fc,
    x_field="X",  # Replace with the name of the X-coordinate field in your CSV
    y_field="Y",  # Replace with the name of the Y-coordinate field in your CSV
    coordinate_system=spatial_ref
)

print(f"Feature class created: {output_fc}")

Feature class created: C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params/C2VSimFG_points_4layers.gdb/Filtered_WCRs_AEM


In [ ]:
# Check out the Spatial Analyst extension
if arcpy.CheckExtension("Spatial") == "Available":
    arcpy.CheckOutExtension("Spatial")
    print("Spatial Analyst extension checked out")
else:
    raise RuntimeError("Spatial Analyst extension is not available or licensed.")

try:
    # Workspace and file paths
    arcpy.env.workspace = "C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params/C2VSimFG_points_4layers.gdb"
    raster_folder = "C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params/"
    input_points = "C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params/C2VSimFG_points_4layers.gdb/Filtered_WCRs_AEM"

    # Validate input points feature class
    if not arcpy.Exists(input_points):
        raise FileNotFoundError(f"Input points feature class '{input_points}' does not exist.")

    # Check and remove the RASTERVALU field if it exists
    fields = [f.name for f in arcpy.ListFields(input_points)]
    if "RASTERVALU" in fields:
        print("Field 'RASTERVALU' exists. Deleting it to avoid conflicts...")
        arcpy.management.DeleteField(input_points, "RASTERVALU")
        print("Field 'RASTERVALU' deleted.")

    # List of rasters to process
    rasters = ['L1_interpolated.tif', 'L2_interpolated.tif', 'L3_interpolated.tif', 'L4_interpolated.tif']

    # Iterate over raster files
    for raster in rasters:
        print(f"Processing {raster}...")

        # Define raster and output feature class
        raster_path = os.path.join(raster_folder, raster)
        if not arcpy.Exists(raster_path):
            print(f"Raster '{raster_path}' does not exist. Skipping...")
            continue

        output_fc = os.path.join(arcpy.env.workspace, f"Extracted_{os.path.splitext(raster)[0]}")

        # Perform ExtractValuesToPoints
        arcpy.sa.ExtractValuesToPoints(
            in_point_features=input_points,
            in_raster=raster_path,
            out_point_features=output_fc,
            interpolate_values="NONE",
            add_attributes="VALUE_ONLY"
        )
        print(f"Values extracted to {output_fc}")

        # Export to CSV
        csv_output = os.path.join(raster_folder, f"{os.path.splitext(raster)[0]}_output.csv")
        fields = [f.name for f in arcpy.ListFields(output_fc) if f.name.lower() != 'shape']
        fields.append('SHAPE@XY')

        with open(csv_output, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(fields[:-1] + ['X', 'Y'])

            with arcpy.da.SearchCursor(output_fc, fields) as cursor:
                for row in cursor:
                    row_list = list(row[:-1])
                    x, y = row[-1]
                    row_list.extend([x, y])
                    writer.writerow(row_list)

        print(f"CSV exported to {csv_output}")

except arcpy.ExecuteError as e:
    print(f"ArcPy ExecuteError: {e}")
except FileNotFoundError as e:
    print(f"FileNotFoundError: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")
finally:
    arcpy.CheckInExtension("Spatial")
    print("Spatial Analyst extension checked in")

Spatial Analyst extension checked out
Field 'RASTERVALU' exists. Deleting it to avoid conflicts...
Field 'RASTERVALU' deleted.
Processing L1_interpolated.tif...
Values extracted to C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params/C2VSimFG_points_4layers.gdb\Extracted_L1_interpolated
CSV exported to C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params/L1_interpolated_output.csv
Processing L2_interpolated.tif...
Values extracted to C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params/C2VSimFG_points_4layers.gdb\Extracted_L2_interpolated
CSV exported to C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params/L2_interpolated_output.csv
Processing L3_interpolated.tif...
Values extracted to C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params/C2VSimFG_points_4layers.gdb\Extracted_L3_interpolated
CSV exported to C:/Users/betebari/Documents/C2VSim_Texture/Aq-Params/L3_interpolated_output.csv
Processing L4_interpolated.tif...
Values extracted to C:/Users/betebari/Documents/C2VSim_Texture/Aq-Pa